<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [2]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [6]:
import re
import spacy


nlp = spacy.load("en_core_web_md") 


In [10]:
def tokenize(doc):
    """
    Takes a document and return a list of tokens in the form of lemmas
    Stop words and punctuation are filtered out
    """
    # token = nlp(doc)
    # return [token.lemma_.strip() for token in doc if (token.is_stop != True) and (token.is_punct != True)]


# tokenize('string')

    tokens = re.sub('[^a-zA-Z ]', '', doc)
    tokens = tokens.lower().split()

    return tokens

In [13]:
tokenize('This is just a test')

['this', 'is', 'just', 'a', 'test']

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews

2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [4]:
yelp.shape

(10000, 9)

In [14]:
# 1. Create a vector representation of the reviews

from sklearn.feature_extraction.text import TfidfVectorizer

# vectorize the text now to learn the vocab
vect = TfidfVectorizer(tokenizer=tokenize, stop_words='english', ngram_range=(1,2), min_df=5, max_df=0.2)

# fit and transform to vectorize and assign to dtm
dtm = vect.fit_transform(yelp['text'])

# then convert to pandas dataframe
dtm = pd.DataFrame(dtm.todense(), columns=vect.get_feature_names())
print(dtm.shape)
dtm.head(4)

(10000, 15313)


,abandoned,abc,ability,able,able accommodate,able come,able eat,able finish,able help,able make,...,z,zach,zen,zero,zero stars,zest,zip,zone,zoo,zucchini
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
# Given the size of the dataset, it will probably be best to use a NearestNeighbors model for this.
from sklearn.neighbors import NearestNeighbors

fake_review = ['This is just a fake review for yelp']

# convert to vector 
fake_review_tfidf = vect.transform(fake_review)
fake_review_tfidf


<1x15313 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [20]:

# Instnatiate NearestNeighbours and mention that we need 10 most similar reviews
nn = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')
# fit on dtm
nn.fit(dtm)

# Query using kneighbours
nn.kneighbors(fake_review_tfidf.todense())

(array([[1.        , 1.        , 1.22336722, 1.24829977, 1.2590022 ,
         1.26853136, 1.27270157, 1.27357344, 1.2797833 , 1.28066041]]),
 array([[6311, 6204, 6019,    0,   14, 7878, 8461,  441, 4607, 3030]]))

In [26]:
# Documents 6311, 6204, 6019,    0,   14, 7878, 8461,  441, 4607, 3030 are all similar
# print all of them here
list = [6311, 6204, 6019,    0,   14, 7878, 8461,  441, 4607, 3030]
for i in list:
    print(f'Document on index {i} is:')
    print(yelp['text'][i])
    print()

Document on index 6311 is:
天氣很熱吃不下東西，今天我點了一個韓國冷面湯、餐後點了甜點，冰沙系列不會太甜膩，覺得店家很用心製作，包含擺盤精緻、佐料衛生，夏日想開胃，這是一個不錯的選擇，服務人員也很敬業，以後會常常來

Document on index 6204 is:
旅行でラスベガスに来ましたがネイルがはげてるのが気になり、探したお店でした。
質問にも丁寧に答えてくれましたし、日本人の方も日本語が話せる方も居て、とても綺麗で居心地のいいお店でした。 
ネイルはちはるさんと言う綺麗な方が丁寧にしてくれとても気に入りました。
予定になかったまつ毛エクステもお願いし、日本ではまだあまりないブラウンカラーのエクステをしてもらい、とても気に入りました。
また是非マッサージなどで伺いたいと思います。

Document on index 6019 is:
I overall liked the atmosphere of this location, however I felt my waiter was really fake, I would ask her questions and she would give me one worded answers, she would ask everyone around the bar if they needed anything, but Skip me. I wouldn't say she was rude, just fake. I felt like she was being a smart Alex. 

The young lady working at the bar should learn to treat all customers the same.

I don't remember her name, she was a Caucasian blonde, working at the bar.

Document on index 0 is:
BEWARE!!! FAKE, FAKE, FAKE....We also own a small business in Los Alamitos, CA and received what looked like a

### Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
Parvi to write here


## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch


### 1. Create a pipeline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 

In [28]:


from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

# create the pipelines components
vect = TfidfVectorizer(
    stop_words='english',
    ngram_range=(1,2), # ana gram and bi gram
    min_df=2, 
    max_df=0.25
    )

svm = LinearSVC() # Support Vector Machine

In [30]:
# Define the pipeline
pipe = Pipeline([
    ('vect', vect),
    ('clf', svm)
])
pipe

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.25, max_features=None,
                                 min_df=2, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words='english', strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
          

In [31]:
yelp.columns

Index(['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text',
       'useful', 'user_id'],
      dtype='object')

In [32]:
# fit using the pipeline first
pipe.fit(yelp['text'], yelp['stars'])

fake_review = ['This is just a fake review for yelp']
# predict
pipe.predict(fake_review)

array([5])

### 2. Tune the entire pipeline with a GridSearch

In [49]:
from sklearn.model_selection import GridSearchCV

# set the parameters
parameters = {
    'vect__max_df': (0.3, 0.5),
    'vect__min_df': (2, 5, 10),
    'vect__max_features': (5000, 20000),
    'clf__penalty': ('l1','l2'),
    'clf__C': (0.1, 0.5, 1., 2.)
}

# do a grid search to find the best parameters 

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(yelp['text'], yelp['stars'])

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:   47.2s
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:  2.2min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=0.25,
                                                        max_features=None,
                                                        min_df=2,
                                                        ngram_range=(1, 2),
                                                        n

In [50]:
grid_search.best_score_

0.6169

In [51]:
grid_search.best_params_

{'clf__C': 0.1,
 'clf__penalty': 'l2',
 'vect__max_df': 0.5,
 'vect__max_features': 20000,
 'vect__min_df': 10}

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [52]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

Learn the vocubalary of the yelp data:

In [54]:
# import and instantiate pandarallel
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
# silent deprecation warnings
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)


# instantiate nlp object
nlp = spacy.load('en_core_web_md', disable = ['parser', 'tagger', 'ner'])

INFO: Pandarallel will run on 20 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [55]:
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [61]:
# clean the text before proceeding

# remove any new line character
yelp['clean_text'] = yelp['text'].apply(lambda x: re.sub('\s+', ' ', x))

# remove non alphanumeric characters
yelp['clean_text'] = yelp['clean_text'].apply(lambda x: re.sub('[^a-zA-Z ]', ' ', x))

# remove extra whitespace and lowercase text
yelp['clean_text'] = yelp['clean_text'].apply(lambda x: ' '.join(x.lower().split()))

# display
yelp['clean_text']

0       beware fake fake fake we also own a small busi...
1       came here for lunch togo service was quick sta...
2       i ve been to vegas dozens of times and had nev...
3       we went here on a night where they closed off ...
4       to stars not bad for the price for lunch senio...
                              ...                        
9995    my family and i were hungry and this subway is...
9996    my wife and i came here with a a couple of fri...
9997    the food was just ok and not anything to brag ...
9998    today s visit is great love and enjoy town squ...
9999    this is the absolute worst place i have ever s...
Name: clean_text, Length: 10000, dtype: object

In [62]:
# convert the text into lemma
yelp['lemma'] = yelp['clean_text'].parallel_apply(lambda x: [token.lemma_ for token in nlp(x) if (token.is_stop != True) and (token.is_punct != True) and (len(token) > 2)])

In [63]:
yelp['lemma']

0       [beware, fake, fake, fake, small, business, lo...
1       [come, lunch, togo, service, quick, staff, fri...
2       [vega, dozen, time, step, foot, circus, circus...
3       [go, night, close, street, party, well, actual...
4       [star, bad, price, lunch, senior, pay, eat, ho...
                              ...                        
9995    [family, hungry, subway, open, hour, guy, work...
9996    [wife, come, couple, friend, sever, excite, po...
9997    [food, brag, food, hot, item, tasty, horrible,...
9998    [today, visit, great, love, enjoy, town, squar...
9999    [absolute, wrong, place, stay, year, life, tim...
Name: lemma, Length: 10000, dtype: object

In [66]:
# create the dictionary
id2word = Dictionary(yelp['lemma'])


Create a bag of words representation of the entire corpus

In [67]:
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in yelp['lemma']]

Your LDA model should be ready for estimation: 

In [68]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=20,
                   num_topics = 10 # You can change this parameter
                  )

In [70]:
# display the print_topics from the model 
from pprint import pprint
pprint(lda.print_topics())

[(0,
  '0.011*"time" + 0.009*"good" + 0.008*"come" + 0.007*"like" + 0.007*"place" + '
  '0.007*"service" + 0.007*"get" + 0.006*"food" + 0.006*"order" + '
  '0.006*"well"'),
 (1,
  '0.016*"place" + 0.009*"food" + 0.009*"time" + 0.008*"service" + '
  '0.008*"great" + 0.008*"like" + 0.007*"come" + 0.007*"good" + 0.006*"order" '
  '+ 0.006*"well"'),
 (2,
  '0.011*"place" + 0.009*"come" + 0.009*"good" + 0.008*"great" + '
  '0.008*"service" + 0.007*"well" + 0.007*"like" + 0.007*"time" + 0.007*"go" + '
  '0.006*"get"'),
 (3,
  '0.011*"good" + 0.011*"food" + 0.010*"place" + 0.009*"like" + 0.009*"great" '
  '+ 0.009*"come" + 0.008*"order" + 0.008*"service" + 0.007*"go" + '
  '0.006*"time"'),
 (4,
  '0.011*"place" + 0.010*"food" + 0.009*"time" + 0.008*"great" + 0.008*"good" '
  '+ 0.007*"order" + 0.007*"like" + 0.006*"come" + 0.006*"service" + '
  '0.005*"well"'),
 (5,
  '0.012*"food" + 0.011*"good" + 0.010*"time" + 0.009*"service" + '
  '0.009*"place" + 0.008*"like" + 0.008*"get" + 0.008*"come"

Create 1-2 visualizations of the results

In [71]:
import pyLDAvis.gensim

In [74]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.006426  0.001878       1        1  11.007192
8     -0.001255  0.001157       2        1  10.472349
2      0.002753  0.002214       3        1  10.155061
5     -0.000285  0.004039       4        1  10.152441
6      0.000013 -0.002868       5        1   9.967757
4      0.002278 -0.002743       6        1   9.845042
9      0.000388 -0.002366       7        1   9.671490
0      0.004696  0.002421       8        1   9.668152
1     -0.000843 -0.001081       9        1   9.656858
3     -0.001320 -0.002651      10        1   9.403659, topic_info=        Term         Freq        Total Category  logprob  loglift
194    great  4146.000000  4146.000000  Default  30.0000  30.0000
135     food  4964.000000  4964.000000  Default  29.0000  29.0000
123    place  5207.000000  5207.000000  Default  28.0000  28.0000
38      good  4651.000000  4651.000000  Default  27.0000  27.0000
43   service  3690.000000  3690.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
456     tell   146.063396  1469.577168  Topic10  -5.7317   0.0554
34      wait   159.598398  1724.206972  Topic10  -5.6431  -0.0158
71       get   189.426972  2822.253457  Topic10  -5.4717  -0.3372
420     know   149.398535  1610.176782  Topic10  -5.7091  -0.0134
561     didn   145.547153  1500.862630  Topic10  -5.7352   0.0308

[859 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
650        1  0.110786     2
650        2  0.083838     2
650        3  0.104798     2
650        4  0.092821     2
650        5  0.140728     2
...      ...       ...   ...
9941       7  0.134772   yuk
9941       8  0.134772   yuk
9941       9  0.112310   yuk
9941      10  0.292007   yuk
17083      7  0.346806  zoey

[2517 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 9, 3, 6, 7, 5, 10, 1, 2, 4])

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Create more visualizations of the LDA results and provide written analysis
* Incorporate RandomizedSearchCV into docoument classification pipeline
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)